---
# combine aggregated spills files
---

In [1]:
import os
import pathlib
import yaml
import numpy
import pandas
import xarray
import h5netcdf
import cartopy.crs
from cartopy import feature
from datetime import datetime
from glob import glob
import time
from matplotlib import pyplot as plt, cm, colors, ticker
#import cartopy.cls
import matplotlib as mpl
mpl.rc('font', size=12)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('legend', fontsize=12)
mpl.rc('axes', titlesize=16)
mpl.rc('axes', labelsize=12)
mpl.rc('figure', titlesize=16)
mpl.rc('text', usetex=False)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')


In [2]:
#pip list --local

### Aggregation functions

In [3]:
def beaching_aggregate(beaching_files,file_idx):
    b = {}
    beach_aggregate = {}
    print([*file_idx])
    # loop through oil types to aggregate
    for sort_tag in [*file_idx]:
        # load first file
        print(beaching_files[file_idx[sort_tag][0]])
        beach_aggregate[sort_tag] = xarray.open_dataset(
            beaching_files[file_idx[sort_tag][0]]
        ).fillna(0)
        nspills=beach_aggregate[sort_tag].attrs['N_spills']
        #files=beach_aggregate[sort_tag].attrs['Filenames'].tolist()
        for idx in range(1,len(file_idx[sort_tag])):
            # load next Xarray
            print(beaching_files[file_idx[sort_tag][idx]])
            b[idx] = xarray.open_dataset(
                beaching_files[file_idx[sort_tag][idx]]
            ).fillna(0)
            # aggregate (n-1) Xarray with (n) Xarray, with special handling
            # for Min. Beaching time 
            BeachTimeMin = beach_aggregate[sort_tag]['BeachTime_Min']
            beach_aggregate[sort_tag] = beach_aggregate[sort_tag]+b[idx]
            # Address special case: Min beaching time
            # Keep time value if it's less than "n" case and real; otherwise, 
            # choose time from nth case, even if NaT.
            beach_aggregate[sort_tag]['BeachTime_Min']=xarray.where(
                numpy.logical_and(
                    BeachTimeMin<b[idx]['BeachTime_Min'],
                    numpy.isreal(BeachTimeMin)),
                    BeachTimeMin.values,
                    b[idx]['BeachTime_Min'].values
            )
            #files+=b[idx].attrs['Filenames'].tolist()
            nspills+=b[idx].attrs['N_spills']
            b[idx].close()
        # update spill count and filename list to reflect full aggregation
        beach_aggregate[sort_tag].attrs['N_spills']=nspills
        #beach_aggregate[sort_tag].attrs['Filenames']=files
        beach_aggregate[sort_tag]['BeachVolume_Mean']=(
            beach_aggregate[sort_tag]['TotalBeachVolume']/beach_aggregate[sort_tag]['BeachPresence']
        )
    return beach_aggregate
        
def surface_aggregate(surface_files,file_idx):
    sfc = {}
    sfc_aggregate = {}
    # loop through oil types to aggregate
    for sort_tag in [*file_idx]:
        # load first file
        print(surface_files[file_idx[sort_tag][0]])
        sfc_aggregate[sort_tag] = xarray.open_dataset(
            surface_files[file_idx[sort_tag][0]]
        ).fillna(0)
        nspills=sfc_aggregate[sort_tag].attrs['N_spills']
        #files=sfc_aggregate[sort_tag].attrs['Filenames'].tolist()
        for idx in range(1,len(file_idx[sort_tag])):
            # load next Xarray
            print(surface_files[file_idx[sort_tag][idx]])
            sfc[idx] = xarray.open_dataset(
                surface_files[file_idx[sort_tag][idx]]
            ).fillna(0)
            # aggregate (n-1) Xarray with (n) Xarray
            sfc_aggregate[sort_tag] = sfc_aggregate[sort_tag]+sfc[idx]
            # concatenate files
            #files+=sfc[idx].attrs['Filenames'].tolist()
            # total up number of runs in aggregation
            nspills+=sfc[idx].attrs['N_spills']
            sfc[idx].close()
        # update spill count and filename list to reflect full aggregation
        sfc_aggregate[sort_tag].attrs['N_spills']=nspills
        #sfc_aggregate[sort_tag].attrs['Filenames']=files  
        sfc_aggregate[sort_tag]['SurfaceVolume_Mean']=(
            sfc_aggregate[sort_tag]['SurfaceVolume_SumSum']/sfc_aggregate[sort_tag]['SurfacePresence']
        )
    return sfc_aggregate        

In [9]:
def create_aggregated_files(output_netcdf_dir, sort_tags):
    """
    output_netcdf_dir: path to aggregated `beaching_*` and `surface_*` files
    sort_tags: oil_types or months, e.g.: oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
    """
    # initialize output dictionaries
    sa={}
    ba={}
    # create list of files to aggregate
    beaching_files = sorted(glob(os.path.join(output_netcdf_dir,"beaching_*")))
    surface_files = sorted(glob(os.path.join(output_netcdf_dir,"surface_*")))
    # create list of indices for grabbing aggregated files by sort_tags
    beaching_file_idx = {}
    surface_file_idx = {}
    for sort_tag in sort_tags: 
        print(sort_tag)
        beaching_file_idx[sort_tag] = [i for i, s in enumerate(beaching_files) if sort_tag in s]
        surface_file_idx[sort_tag] = [i for i, s in enumerate(surface_files) if sort_tag in s]
    # aggregate     
    ba = beaching_aggregate(beaching_files,beaching_file_idx)
    sa = surface_aggregate(surface_files,surface_file_idx)
    
    for sort_tag in sort_tags: 
        try:
            os.makedirs(output_netcdf_dir/'complete')
        except:
            print(f'{output_netcdf_dir}/complete already exists')
        ba[sort_tag].to_netcdf(
            output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc', engine='h5netcdf'
        )
        sa[sort_tag].to_netcdf(
            output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc', engine='h5netcdf'
        )
    return ba,sa

In [5]:
# def load_aggregated_files(output_netcdf_dir, sort_tags):
#     """
#     output_netcdf_dir: path to aggregated `beaching_*` and `surface_*` files
#     sort_tags: oil_types or months, e.g.: oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
#     """
#     # initialize output dictionaries
#     sa={}
#     ba={}
#     # create list of files to aggregate
#     beaching_files = sorted(glob(os.path.join(output_netcdf_dir,"beaching_*")))
#     surface_files = sorted(glob(os.path.join(output_netcdf_dir,"surface_*")))
#     # create list of indices for grabbing aggregated files by sort_tags
#     beaching_file_idx = {}
#     surface_file_idx = {}
#     # *** beaching files ***
#     for sort_tag in sort_tags: 
#         print(sort_tag)
#         beaching_file_idx[sort_tag] = [i for i, s in enumerate(beaching_files) if sort_tag in s]
#         beaching_path = output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc'
#         try:
#             ba[sort_tag]=xarray.open_dataset(beaching_path)
#         except:
#             print('Not found: ', output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc')
#             ba = beaching_aggregate(beaching_files,beaching_file_idx)
#             for sort_tag in sort_tags: 
#                 try:
#                     os.makedirs(output_netcdf_dir/'complete')
#                 except:
#                     print(f'{output_netcdf_dir}/complete already exists')
#                 ba[sort_tag].to_netcdf(
#                     output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc', engine='h5netcdf'
#                 )
#                 # sa[sort_tag].to_netcdf(
#                 #     output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc', engine='h5netcdf'
#                 # )
#             return ba
#     # *** surface files ***
#     for sort_tag in sort_tags: 
#         surface_file_idx[sort_tag] = [i for i, s in enumerate(surface_files) if sort_tag in s]
#         surface_path = output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc'
#         try:
#             sa[sort_tag]=xarray.open_dataset(surface_path)
#         except:
#             print('Not found: ', output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc')
#             sa = surface_aggregate(surface_files,surface_file_idx) 
#             for sort_tag in sort_tags: 
#                 try:
#                     os.makedirs(output_netcdf_dir/'complete')
#                 except:
#                     print(f'{output_netcdf_dir}/complete already exists')
#                 sa[sort_tag].to_netcdf(
#                     output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc', engine='h5netcdf'
#                 )
#             return sa
#     # See if fully aggregated files exist already and create them if they don't
#     # *** surface ***
#     # beaching_path = output_netcdf_dir/'complete'/f'beaching_{sort_tags[0]}_complete.nc'
#     # surface_path = output_netcdf_dir/'complete'/f'surface_{sort_tags[0]}_complete.nc'    
#     # try:
#     #     sa[sort_tags[0]]=xarray.open_dataset(surface_path)
#     # except:
#     #     print('Not found: ', output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc')
#     #     sa = surface_aggregate(surface_files,surface_file_idx) 
#     #     for sort_tag in sort_tags: 
#     #         try:
#     #             os.makedirs(output_netcdf_dir/'complete')
#     #         except:
#     #             print(f'{output_netcdf_dir}/complete already exists')
#     #         sa[sort_tag].to_netcdf(
#     #             output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc', engine='h5netcdf'
#     #         )
#     #     return sa
#     # # *** beaching ***
#     # try:
#     #     ba[sort_tags[0]]=xarray.open_dataset(beaching_path)
#     # except:
#     #     print('Not found: ', output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc')
#     #     ba = beaching_aggregate(beaching_files,beaching_file_idx)
#     #     for sort_tag in sort_tags: 
#     #         try:
#     #             os.makedirs(output_netcdf_dir/'complete')
#     #         except:
#     #             print(f'{output_netcdf_dir}/complete already exists')
#     #         ba[sort_tag].to_netcdf(
#     #             output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc', engine='h5netcdf'
#     #         )
#     #         sa[sort_tag].to_netcdf(
#     #             output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc', engine='h5netcdf'
#     #         )
#     #     return ba
    
#     # Load files if they exist
#     for sort_tag in sort_tags:
#         beaching_path = output_netcdf_dir/'complete'/f'beaching_{sort_tag}_complete.nc'
#         surface_path = output_netcdf_dir/'complete'/f'surface_{sort_tag}_complete.nc'    
#         print(f'loading {surface_path}') 
#         print(f'loading {beaching_path}') 
#         sa[sort_tag]=xarray.open_dataset(surface_path)
#         ba[sort_tag]=xarray.open_dataset(beaching_path)
#     return sa, ba

### load aggregation by oil

In [10]:
output_netcdf_dir_oil=pathlib.Path('/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0') 
oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
[sa_oil,ba_oil]=create_aggregated_files(output_netcdf_dir_oil, oil_types)

akns
bunker
dilbit
jet
diesel
gas
other
['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_akns_0-200.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_0-200.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_1000-1200.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_1200-1400.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_1400-1600.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_1600-1800.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_1800-2000.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_200-400.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30_fillna0/beaching_bunker_2000-2200.nc
/scratch/rmueller/MIDOSS/Results/ByOilType/Spi

### load aggregation by months

In [11]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
output_netcdf_dir_months=pathlib.Path('/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0') 
[sa_month,ba_month]=create_aggregated_files(output_netcdf_dir_months, months)

Jan
Feb
Mar
Apr
May
Jun
Jul
Aug
Sep
Oct
Nov
Dec
['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Jan_0-200.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Jan_200-400.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Jan_400-600.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Jan_600-800.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Feb_0-200.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Feb_200-400.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Feb_400-600.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Feb_600-800.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beaching_Mar_0-200.nc
/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30_fillna0/beachin

In [ ]:
output_netcdf_dir=pathlib.Path('/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30') 
for oil in oil_types:
    try:
        test = xarray.open_dataset(
            output_netcdf_dir/'complete'/f'beaching_{oil}_complete.nc'
        )
    except:
        print('?')

### load runlist used to create aggregated files 

In [ ]:


with yaml_file.open("rt") as f:
    run_paths = yaml.safe_load(f) 

##### create list of aggregated files by oil type

In [ ]:
output_netcdf_dir=pathlib.Path('/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30') 
beaching_files = sorted(glob(os.path.join(output_netcdf_dir,"beaching_*")))
surface_files = sorted(glob(os.path.join(output_netcdf_dir,"surface_*")))

In [ ]:
ds = xarray.open_dataset(surface_files[0])

# Get indices of files by oil-types in order to aggregate the aggregated files

In [ ]:
beaching_file_idx = {}
surface_file_idx = {}
oil_types = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
for oil in oil_types: 
    beaching_file_idx[oil] = [i for i, s in enumerate(beaching_files) if oil in s]
    print(beaching_file_idx[oil])
for oil in oil_types: 
    surface_file_idx[oil] = [i for i, s in enumerate(surface_files) if oil in s]   

In [ ]:
# try to load completely aggregated files or create them if needed
# *** surface ***
try:
    sa[oil]=xarray.open_dataset(output_netcdf_dir/'complete'/f'surface_{oil}_complete.nc')
except:
    sa = surface_aggregate(surface_files,beaching_file_idx) 
    for oil in oil_types: 
        try:
            os.makedirs(output_netcdf_dir/'complete')
        except:
            print(f'{output_netcdf_dir}/complete already exists')
        sa[oil].to_netcdf(
            output_netcdf_dir/'complete'/f'surface_{oil}_complete.nc', engine='h5netcdf'
        )
# *** beaching ***
try:
    ba[oil]=xarray.open_dataset(output_netcdf_dir/'complete'/f'beaching_{oil}_complete.nc')
except:
    ba = beaching_aggregate(beaching_files,beaching_file_idx)
    for oil in oil_types: 
        try:
            os.makedirs(output_netcdf_dir/'complete')
        except:
            print(f'{output_netcdf_dir}/complete already exists')
        ba[oil].to_netcdf(
            output_netcdf_dir/'complete'/f'beaching_{oil}_complete.nc', engine='h5netcdf'
        )
        sa[oil].to_netcdf(
            output_netcdf_dir/'complete'/f'surface_{oil}_complete.nc', engine='h5netcdf'
        )

In [ ]:
[*sa]

### Check to make sure that spill volumes between 0-24hrs, 24-72hrs, and 72-168hrs sum up to equal total spill volumes

In [ ]:
print('*** BEACHING PRESENCE *** ')
for oil in ['diesel','bunker']:
    ba_presence_diff = (ba[oil]['BeachPresence']-
                        ba[oil]['BeachPresence_24h']-
                        ba[oil]['BeachPresence_24h_to_72h']-
                        ba[oil]['BeachPresence_72h_to_168h']
                       )
    print(f'{oil}: {ba_presence_diff.sum().values.item()}')
    
print('*** SURFACE PRESENCE *** ')
for oil in ['diesel','bunker']:
    sa_presence_diff = (sa[oil]['SurfacePresence']-
                        sa[oil]['SurfacePresence_24h']-
                        sa[oil]['SurfacePresence_24h_to_72h']-
                        sa[oil]['SurfacePresence_72h_to_168h']
                       )
    print(f'{oil}: {sa_presence_diff.sum().values.item()}')
    
print('*** SURFACE VOLUME *** ')
for oil in ['diesel','bunker']:
    sa_presence_diff = (sa[oil]['SurfaceVolume_SumSum']-
                        sa[oil]['SurfaceVolume_SumSum_24h']-
                        sa[oil]['SurfaceVolume_SumSum_24h_to_72h']-
                        sa[oil]['SurfaceVolume_SumSum_72h_to_168h']
                       )
    print(f'{oil}: {sa_presence_diff.sum().values.item()}')

### Check individual aggregation files.  Diesel is aggregated; bunker is not.  

In [ ]:
b={}
for idx,name in enumerate(beaching_files):
    b[idx]=xarray.open_dataset(name)

In [ ]:
name.split('/')[-1:]

In [ ]:
for idx in [*b]:
    ba_presence_diff = (b[idx]['BeachPresence']-
                        b[idx]['BeachPresence_24h']-
                        b[idx]['BeachPresence_24h_to_72h']-
                        b[idx]['BeachPresence_72h_to_168h']
                       )
    print(f'{beaching_files[idx].split("/")[-1:]}: {ba_presence_diff.sum().values.item()}')

### Plot results

In [ ]:
plot_types = ['SurfacePresence','SurfaceVolume_SumSum','SurfaceVolume_Mean']

In [ ]:
mx={}
for plot_type in plot_types:
    mx[plot_type]=0
    for idx,oil in enumerate(oil_types):
        mx[plot_type] = max(mx[plot_type], sa[oil][plot_type].max().values.item())

In [ ]:
mx

In [ ]:
sa['all']=xarray.zeros_like(sa['akns'])
ba['all']=xarray.zeros_like(ba['akns'])
for oil in oil_types:
    ba['all']+=ba[oil]
    sa['all']+=sa[oil]

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(6, 8), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
cs=axs.contourf(sa['all']['SurfacePresence'])
fig.colorbar(cs, ax=axs, shrink=0.9, orientation='vertical')
axs.set_title('Surface Presence (all oil types)')    

In [ ]:
cs={}
plot_type='SurfacePresence'
fig, axs = plt.subplots(1, 7, figsize=(16, 8), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
for idx,oil in enumerate(oil_types):
    cs[idx]=axs[idx].contourf(
        sa[oil][plot_type])
    axs[idx].set_title(oil)
    fig.colorbar(cs[idx], ax=axs[idx], shrink=0.9, orientation='vertical')
    if idx==0:
        axs[idx].set_ylabel(plot_type)
    axs[idx].set_xticks([])
    axs[idx].set_yticks([])
fig, axs = plt.subplots(1, 7, figsize=(16, 8), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
for idx,oil in enumerate(oil_types):
    cs[idx]=axs[idx].contourf(
        sa[oil][plot_type], 
        50, vmin=0,vmax=mx[plot_type]/1.5
    )
    axs[idx].set_title(oil)
    fig.colorbar(cs[idx], ax=axs[idx], shrink=0.9, orientation='vertical')
    if idx==0:
        axs[idx].set_ylabel(plot_type)
    axs[idx].set_xticks([])
    axs[idx].set_yticks([])

### By months

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
output_netcdf_dir=pathlib.Path('/scratch/rmueller/MIDOSS/Results/ByMonth/SpillTime_00-30') 
beaching_files = sorted(glob(os.path.join(output_netcdf_dir,"beaching_*")))
surface_files = sorted(glob(os.path.join(output_netcdf_dir,"surface_*")))
beaching_file_idx = {}
surface_file_idx = {}
for month in months: 
    beaching_file_idx[month] = [i for i, s in enumerate(beaching_files) if month in s]
    print(beaching_file_idx[month])
for mont in months: 
    surface_file_idx[month] = [i for i, s in enumerate(surface_files) if month in s]
    
ba = beaching_aggregate(beaching_files,beaching_file_idx)
sa = surface_aggregate(surface_files,beaching_file_idx)

# Plot up results 
cs={}
plot_type='SurfacePresence'
fig, axs = plt.subplots(1, 7, figsize=(16, 8), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
for idx,month in enumerate(months[:6]):
    cs[idx]=axs[idx].contourf(
        sa[month][plot_type])
    axs[idx].set_title(month)
    fig.colorbar(cs[idx], ax=axs[idx], shrink=0.9, orientation='vertical')
    if idx==0:
        axs[idx].set_ylabel(plot_type)
    axs[idx].set_xticks([])
    axs[idx].set_yticks([])

In [ ]:
# lev_exp = np.arange(np.floor(np.log10(z.min())-1),
#                    np.ceil(np.log10(z.max())+1))
# levs = np.power(10, lev_exp)
# cs = ax.contourf(X, Y, z, levs, norm=colors.LogNorm())


In [ ]:
ds = xarray.open_dataset('/scratch/rmueller/MIDOSS/Results/ByOilType/SpillTime_00-30/mohid_diesel_0-200.nc')

In [ ]:
[*ds]

In [ ]:
ds.info

In [ ]:
ds['SurfaceVolumeSum'][0,:,:].plot.contourf()

In [ ]:
ds['SurfaceVolume_SumSum_ln'].plot.contourf()

In [ ]:
[*ds]

In [ ]:
ds['SurfacePresence'].fillna(0)
ds['SurfacePresence'].plot.contourf()

In [ ]:
cs={}
fig, axs = plt.subplots(1, 3, figsize=(8, 4), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
cs[0]=axs[0].contourf(numpy.exp(MeanVol))
cs[1]=axs[1].contourf(numpy.exp(sa['SurfaceVolume_SumSum']))
cs[2]=axs[2].contourf(sa['SurfacePresence'])
axs[0].set_title('Volume/Runs')
axs[1].set_title('$\sum$ Volume')
axs[2].set_title('Number Runs \n(Presence)')
for ind in numpy.arange(0,3):
    fig.colorbar(cs[ind], ax=axs[ind], shrink=0.9, orientation='vertical')
    axs[ind].set_xticks([])
    axs[ind].set_yticks([])

### Review beaching time calc. 

In [ ]:
mesh_mask = xarray.open_dataset("/home/rmueller/projects/def-allen/rmueller/MEOPAR/grid/mesh_mask202108.nc")
rotated_crs = cartopy.crs.RotatedPole(pole_longitude=120.0, pole_latitude=63.75)
plain_crs = cartopy.crs.PlateCarree()

In [ ]:
x1 = xarray.open_dataset(run_paths['akns'][0], engine='h5netcdf')

In [ ]:
nanosecond_to_hour = 1e-9/3600
spill_start=x1.time[0]
one_day = numpy.timedelta64(24,'h')
dt_beaching=(x1.Beaching_Time-spill_start)*nanosecond_to_hour
dt_surface=(x1.Oil_Arrival_Time-spill_start)*nanosecond_to_hour
print(f'Max beaching time: {dt_beaching.max().values.item()} hrs')
print(f'Max surface time: {dt_surface.max().values.item()} hrs')

In [ ]:
%matplotlib inline    
fig, axs = plt.subplots(1, 2, figsize=(8, 4), 
                        gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
cs0=axs[0].contourf(dt_beaching)
cs1=axs[1].contourf(dt_surface)
axs[0].set_title(f'Beaching time')
axs[1].set_title(f'Surface time')
fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='vertical')
fig.colorbar(cs0, ax=axs[1], shrink=0.9, orientation='vertical')
for ind in numpy.arange(0,2):
    axs[ind].set_xticks([])
    axs[ind].set_xlim(150,350)
    axs[ind].set_yticks([])
    axs[ind].set_ylim(200,450)

In [ ]:
# import scipy
# %matplotlib inline    
# fig, axs = plt.subplots(1, 2, figsize=(8, 4), 
#                         gridspec_kw={'wspace': 0.25, 'hspace': 0.1},
#                        subplot_kw={"projection": rotated_crs, "facecolor": "white"})
# for ind in numpy.arange(0,2):
#     axs[ind].add_feature(feature.GSHHSFeature('intermediate', edgecolor='k', facecolor='#8b7765'), zorder=1)
# # cs0=axs[0].contourf(dt_beaching)
# # cs1=axs[1].contourf(dt_surface)
# axs[0].set_title(f'Beaching time')
# axs[1].set_title(f'Surface time')
# fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='vertical')
# fig.colorbar(cs0, ax=axs[1], shrink=0.9, orientation='vertical')
# for ind in numpy.arange(0,2):
#     axs[ind].set_xticks([])
#     axs[ind].set_xlim(150,350)
#     axs[ind].set_yticks([])
#     axs[ind].set_ylim(200,450)

In [ ]:
vol3d=x1.OilWaterColumnOilVol_3D.isel({'grid_z': 39})
test = vol3d.where(vol3d>15,0).loc[dict(
                time=slice(x1.time[0],x1.time[0]+numpy.timedelta64(24,'h'))
                )]

## Plot restults from test run 

In [ ]:
geomean_vars = ['SurfaceConcentration_SumSum','SurfaceVolume_MaxSum', 
            'SurfaceVolume_SumSum','SurfaceVolume_SumSum_24h',
            'SurfaceVolume_SumSum_24h_to_72h','SurfaceVolume_SumSum_72h_to_168h',
            'TotalBeachVolume']
mean_vars = ['BeachTime_Sum']

In [ ]:
ba = xarray.open_dataset('/scratch/rmueller/MIDOSS/Results/test/beaching_diesel_0-500.nc', engine='h5netcdf')
sa = xarray.open_dataset('/scratch/rmueller/MIDOSS/Results/test/surface_diesel_0-500.nc', engine='h5netcdf')
for var in [*ba]:
    fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    cs0=axs.contourf(ba[var])
    axs.set_title(f'{var}')
    fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')
for var in [*sa]:
    fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    cs0=axs.contourf(sa[var])
    axs.set_title(f'{var}')      
    fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')


# for var in [*ba]:
#     fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
#     if var not in mean_vars:
#         cs0=axs.contourf(ba[var])
#         axs.set_title(f'{var}')
#         if var in geomean_vars:
#             cs0=axs.contourf(
#             numpy.exp(ba[var]/ba.attrs['N_spills'])
#             )
#             axs.set_title('Bunker \nGeometric Mean Volume (m$^3$)')
#     else:
#         cs0=axs.contourf(
#             ba[var]/ba.attrs['N_spills']
#             )
#         axs.set_title('Avg Beaching Time (hours)')
#     fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')
# for var in [*sa]:
#     fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
#     if var not in geomean_vars:
#         cs0=axs.contourf(sa[var])
#         axs.set_title(f'{var}')
#     else:
#         cs0=axs.contourf(
#             numpy.exp(sa[var]/sa.attrs['N_spills'])
#         )
#         if var == 'SurfaceConcentration_SumSum':
#             axs.set_title('Geometric Mean Conc. (Kg/m$^3$)')
#         else:
#             axs.set_title(f'{var}')
            
#     fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')
ba.close()
sa.close()

In [ ]:
MeanVol = sa['SurfaceVolume_SumSum']/sa['SurfacePresence']
cs={}
fig, axs = plt.subplots(1, 3, figsize=(8, 4), gridspec_kw={'wspace': 0.5, 'hspace': 0.5})
cs[0]=axs[0].contourf(numpy.exp(MeanVol))
cs[1]=axs[1].contourf(numpy.exp(sa['SurfaceVolume_SumSum']))
cs[2]=axs[2].contourf(sa['SurfacePresence'])
axs[0].set_title('Volume/Runs')
axs[1].set_title('$\sum$ Volume')
axs[2].set_title('Number Runs \n(Presence)')
for ind in numpy.arange(0,3):
    fig.colorbar(cs[ind], ax=axs[ind], shrink=0.9, orientation='vertical')
    axs[ind].set_xticks([])
    axs[ind].set_yticks([])

In [ ]:
ba = xarray.open_dataset('/scratch/rmueller/MIDOSS/Results/beaching_diesel_0-50.nc', engine='h5netcdf')
sa = xarray.open_dataset('/scratch/rmueller/MIDOSS/Results/surface_diesel_0-50.nc', engine='h5netcdf')

for var in [*ba]:
    fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    if var not in mean_vars:
        cs0=axs.contourf(ba[var])
        axs.set_title(f'{var}')
        if var in geomean_vars:
            cs0=axs.contourf(
            numpy.exp(ba[var]/ba.attrs['N_spills'])
            )
            axs.set_title('Bunker \nGeometric Mean Volume (m$^3$)')
    else:
        cs0=axs.contourf(
            ba[var]/ba.attrs['N_spills']
            )
        axs.set_title('Avg Beaching Time (hours)')
    fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')
for var in [*sa]:
    fig, axs = plt.subplots(1, 1, figsize=(4, 2), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    if var not in geomean_vars:
        cs0=axs.contourf(sa[var])
        axs.set_title(f'{var}')
    else:
        cs0=axs.contourf(
            numpy.exp(sa[var]/sa.attrs['N_spills'])
        )
        if var == 'SurfaceConcentration_SumSum':
            axs.set_title('Geometric Mean Conc. (Kg/m$^3$)')
        else:
            axs.set_title(f'{var}')
            
    fig.colorbar(cs0, ax=axs, shrink=0.9, orientation='vertical')
    

# Aggregate Beaching and Surface files

In [ ]:
# Create dictionary with indices of file to aggregate by oil type
# (Identifying indices will eventually be coded in)
beaching_file_idx = {}
beaching_file_idx['bunker']=[0,1]
beaching_file_idx['diesel']=[2,3]

In [ ]:
ba = beaching_aggregate(beaching_files,beaching_file_idx)
sa = surface_aggregate(surface_files,beaching_file_idx) #file index same for beaching in this case

In [ ]:
[*sa['bunker']]

In [ ]:
[*ba['bunker']]

In [ ]:
for var in [*ba['bunker']]:
    fig, axs = plt.subplots(1, 2, figsize=(8, 4), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    if var not in mean_vars:
        cs0=axs[0].contourf(ba['bunker'][var])
        cs1=axs[1].contourf(ba['diesel'][var])
        axs[0].set_title(f'Bunker \n{var}')
        axs[1].set_title(f'Diesel \n{var}')
        if var in geomean_vars:
            cs0=axs[0].contourf(
            numpy.exp(ba['bunker'][var]/ba['bunker'].attrs['N_spills'])
            )
            cs1=axs[1].contourf(
            numpy.exp(ba['diesel'][var]/ba['diesel'].attrs['N_spills'])
            )
            axs[0].set_title('Bunker \nGeometric Mean Volume (m$^3$)')
            axs[1].set_title('Diesel \nGeometric Mean Volume (m$^3$)')
    else:
        cs0=axs[0].contourf(
            ba['bunker'][var]/ba['bunker'].attrs['N_spills']
            )
        cs1=axs[1].contourf(
        ba['diesel'][var]/ba['diesel'].attrs['N_spills']
        )
        axs[0].set_title('Bunker \nAvg Beaching Time (hours)')
        axs[1].set_title('Diesel \nAvg Beaching Time (hours)')
    fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='vertical')
    fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='vertical')
for var in [*sa['bunker']]:
    fig, axs = plt.subplots(1, 2, figsize=(8, 4), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
    if var not in geomean_vars:
        cs0=axs[0].contourf(sa['bunker'][var])
        cs1=axs[1].contourf(sa['diesel'][var])
        axs[0].set_title(f'Bunker \n{var}')
        axs[1].set_title(f'Diesel \n{var}')
    else:
        cs0=axs[0].contourf(
        numpy.exp(sa['bunker'][var]/sa['bunker'].attrs['N_spills'])
        )
        cs1=axs[1].contourf(
        numpy.exp(sa['diesel'][var]/sa['diesel'].attrs['N_spills'])
        )
        if var == 'SurfaceConcentration_SumSum':
            axs[0].set_title('Bunker \nGeometric Mean Conc. (Kg/m$^3$)')
            axs[1].set_title('Diesel \nGeometric Mean Conc (Kg/m$^3$)')
        else:
            axs[0].set_title(f'Bunker \n{var}')
            axs[1].set_title(f'Diesel \n{var}')

    fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='vertical')
    fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='vertical')

# Plot Beaching

In [ ]:
vmin = ba['bunker']['TotalBeachVolume'].where(ba['bunker']['TotalBeachVolume']>0).min()
vmax = ba['bunker']['TotalBeachVolume'].where(ba['bunker']['TotalBeachVolume']>0).max()
levels = numpy.linspace(numpy.exp(vmin), 1e7, 5)
levels

In [ ]:
ba['bunker'].attrs['N_spills']

In [ ]:

fig, axs = plt.subplots(1, 3, figsize=(12, 6), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
cs0=axs[0].contourf(ba['bunker']['BeachPresence'])
cs1=axs[1].contourf(ba['bunker']['BeachPresence_24h'])
cs2=axs[2].contourf(
    numpy.exp(ba['bunker']['TotalBeachVolume']/ba['bunker'].attrs['N_spills']),
    #locator=ticker.LogLocator(),
    levels=levels
)
axs[0].set_title('100 Aggregated files\nBeachPresence')
axs[1].set_title('100 Aggregated files\nBeachPresence_24h_to_72h')
axs[2].set_title('100 Aggregated files\nGeometric Mean Volume (m$^3$)')
fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='horizontal')
fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='horizontal')
fig.colorbar(cs2, ax=axs[2], shrink=0.9, orientation='horizontal')

In [ ]:
from matplotlib import pyplot as plt, cm, colors, ticker
fig, axs = plt.subplots(1, 3, figsize=(12, 6), gridspec_kw={'wspace': 0.25, 'hspace': 0.1})
cs0=axs[0].contourf(ba['bunker']['BeachPresence'])
cs1=axs[1].contourf(ba['bunker']['BeachPresence_24h'])
#cs2=axs[2].contourf(numpy.log(ba['bunker']['TotalBeachVolume'].where(ba['bunker']['TotalBeachVolume']>0)))
cs2=axs[2].contourf(ba['bunker']['TotalBeachVolume'])
axs[0].set_title('100 Aggregated files\nBeachPresence')
axs[1].set_title('100 Aggregated files\nBeachPresence_24h_to_72h')
axs[2].set_title('100 Aggregated files\n$\ln$(TotalBeachVolume)')
fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='horizontal')
fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='horizontal')
fig.colorbar(cs2, ax=axs[2], shrink=0.9, orientation='horizontal')

In [ ]:
print('--------')
print('BEACHING')
print('--------')
for oil in ['bunker','diesel']:
    print('***',oil,'***')
    for variable in [*ba[oil]]:
        variable
        print(variable,': ', numpy.nanmin(ba[oil][variable].values).item(),
              '-', ba[oil][variable].max().values.item())
print('-------')
print('SURFACE')
print('-------')
for oil in ['bunker','diesel']:
    print('***',oil,'***')
    for variable in [*sa[oil]]:
        variable
        print(variable,': ', sa[oil][variable].min().values.item(),
              '-', sa[oil][variable].max().values.item())

# Surface

In [ ]:
surface_files

In [ ]:
for oil in [*beaching_file_idx]:
    print(oil)

In [ ]:
[*sa]

In [ ]:
sfc = {}
sfc[0]=xarray.open_dataset(surface_files[file_idx['bunker'][0]])
sfc[1]=xarray.open_dataset(surface_files[file_idx['bunker'][1]])

In [ ]:
[*sfc[0]]

###  Aggregate n and (n-1) file

In [ ]:
sfc_aggregated = sfc[0] + sfc[1]
#sfc_aggregated = sfc[1]

In [ ]:
for variable in [*sfc_aggregated]:
    variable
    print(variable,': ', numpy.exp(sfc_aggregated[variable].min().values.item()),
          '-', sfc_aggregated[variable].max().values.item())

In [ ]:
limits={}
limits['SurfacePresence']=(0,20)


In [ ]:
for var_idx,variable in enumerate([*sfc[0]]):
    if variable != 'SurfaceDissolution_SumSum':
        fig,axs = plt.subplots(
            1, 3, figsize=(18, 9), 
            subplot_kw={"facecolor": "white"}
        )
        fig.subplots_adjust(hspace=0.5)
        if variable=='SurfacePresence':
            cs0=axs[0].contourf(sfc[0][variable], 
                levels=numpy.linspace(limits[variable][0],limits[variable][1],20))
            cs1=axs[1].contourf(sfc[1][variable], 
                levels=numpy.linspace(limits[variable][0],limits[variable][1],20))
            cs2=axs[2].contourf(numpy.exp(sfc_aggregated[variable]),norm=colors.LogNorm())
        else:
            cs0=axs[0].contourf(sfc[0][variable])
            cs1=axs[1].contourf(sfc[1][variable])
            cs2=axs[2].contourf(sfc_aggregated[variable])
        axs[0].set_ylabel(variable)
        # axs[1].set_title('BeachPresence_24h_to_72h')
        # axs[2].set_title('TotalBeachVolume')
        fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='horizontal')
        fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='horizontal')
        fig.colorbar(cs2, ax=axs[2], shrink=0.9, orientation='horizontal')

## Compare two-ways of aggregating surface presence

In [ ]:
sfcPresence_aggregated = sfc[0]['SurfacePresence'] + sfc[1]['SurfacePresence']
sum(sum(sfc_aggregated[variable] - sfcPresence_aggregated)).values.item()

In [ ]:
variable = 'SurfacePresence'
fig,axs = plt.subplots(
    1, 2, figsize=(9, 9), 
    subplot_kw={"facecolor": "white"}
)
fig.subplots_adjust(hspace=0.5)
cs0=axs[0].contourf(sfcPresence_aggregated, 
        levels=numpy.linspace(limits[variable][0],limits[variable][1],10))
cs1=axs[1].contourf(sfc_aggregated[variable],
        levels=numpy.linspace(limits[variable][0],limits[variable][1],10))

axs[0].set_title('SurfacePresence \n(direct addition)')
axs[1].set_title('SurfacePresence \n(Xarray addition)')
cbar1=fig.colorbar(cs0, ax=axs[0], shrink=0.9, orientation='horizontal')
cbar2=fig.colorbar(cs1, ax=axs[1], shrink=0.9, orientation='horizontal')
labels = numpy.arange(limits[variable][0],limits[variable][1],2)
cbar1.set_ticks(labels)
cbar2.set_ticks(labels)
#cbar.ax.set_xticklabels(labels, rotation=45)

## Evaluate surface dissolution, concentration and volume in the top 5 largest spills in these aggregations 

In [ ]:
First100 = {}
for oil in ['diesel','bunker']:
    First100[oil] = run_paths[oil][0:100]

In [ ]:
First100['diesel'][0]

In [ ]:
First100['diesel'][0].split('/')[0:-1]

In [ ]:
First100['diesel'][0].split('_')[-3]

In [ ]:
"/".join(First100['diesel'][0].split("/")[:-1]) 

### Get spill volumes and sort spills to get 5 largest spills by oil type

In [ ]:
big5_spills={}
for oil in ['diesel','bunker']:
    volume = {}
    volume[oil]={}
    volume[oil]['volume']=[]
    for idx,ncfile in enumerate(First100[oil]):
        directory = "/".join(ncfile.split("/")[:-1]) 
        oil_tag = First100[oil][idx].split('_')[-3]
        # Open the Lagrangian file used for spill
        Lagrangian_file = directory+'/Lagrangian_'+oil_tag+'.dat'
        with open(Lagrangian_file, 'r') as f:
            for line in f:
                if 'POINT_VOLUME              :' in line: 
                    # select spill volume quantity after ':'
                    spillvolume = line.split(':')[-1].split('\n')[0]
                    # convert to float for "sort_values" to work properly
                    volume[oil]['volume'].append(float(spillvolume)) 
        big5_spills[oil] = pandas.DataFrame.from_dict(
            volume[oil]).sort_values('volume',ascending=False)[0:5]

In [ ]:
big5_spills['diesel']

### Use indices from big5_spills to open and plot output from the top 5 spills

In [ ]:
big5_spills[oil].index.values

In [ ]:
%%time
TopSpills = pandas.read_csv('/home/rmueller/projects/def-allen/rmueller/MIDOSS/analysis-rachael/notebooks/monte_carlo_dev/TopSpills_df.csv')

# ## Generate "TopSpills_df.csv"
# ## This section takes 10 minutes on 5210 Mb
# vars2plot= ['Dissolution_3D','OilConcentration_3D','OilWaterColumnOilVol_3D']
# oil_type=[]
# spill_index=[]
# spill_volume=[]
# #variable_list = []
# mn={}
# mx={}
# for variable in vars2plot:
#     mn[variable] = []
#     mx[variable] = []

# for oil in ['diesel','bunker']:
#     # loop through indices of the 5 largest spills in the first 100 spill files
#     for idx in big5_spills[oil].index.values:
#         spill_xr=xarray.open_dataset(First100[oil][idx])
#         oil_type.append(oil)
#         spill_index.append(idx)
#         spill_volume.append(big5_spills[oil]['volume'][idx])
#         for variable in vars2plot:
#             print(f'[{oil}-{idx}-{variable}]')     
#             mn[variable].append(spill_xr[variable][:,39,:,:].min().values.item())
#             mx[variable].append(spill_xr[variable][:,39,:,:].max().values.item())
#         spill_xr.close()
# TopSpills = {'oil_type':oil_type, 'spill_volume':spill_volume,
#     'MinSfcDiss':mn['Dissolution_3D'], 'MaxSfcDiss':mx['Dissolution_3D'],
#     'MinSfcConc':mn['OilConcentration_3D'], 'MaxSfcConc':mx['OilConcentration_3D'],
#     'MinSfcVol':mn['OilWaterColumnOilVol_3D'], 'MaxSfcVol':mx['OilWaterColumnOilVol_3D']}
# TopSpills_df = pandas.DataFrame(TopSpills)
# TopSpills_df.to_csv('TopSpills_df.csv')

In [ ]:
TopSpills

In [ ]:
Conc2Vol = TopSpills['MaxSfcVol']/TopSpills['MaxSfcConc']

In [ ]:
TopSpills[TopSpills['oil_type']=='diesel'].plot.scatter(x='MaxSfcVol',y='MaxSfcConc',s=60, c='purple')

In [ ]:
TopSpills[TopSpills['oil_type']=='bunker'].plot.scatter(x='MaxSfcVol',y='MaxSfcConc', 
                                                        s=60, c='green')

## Get top 50 spills

In [ ]:
big50_spills={}
for oil in ['diesel','bunker']:
    volume = {}
    volume[oil]={}
    volume[oil]['volume']=[]
    for idx,ncfile in enumerate(First100[oil]):
        directory = "/".join(ncfile.split("/")[:-1]) 
        oil_tag = First100[oil][idx].split('_')[-3]
        # Open the Lagrangian file used for spill
        Lagrangian_file = directory+'/Lagrangian_'+oil_tag+'.dat'
        with open(Lagrangian_file, 'r') as f:
            for line in f:
                if 'POINT_VOLUME              :' in line: 
                    # select spill volume quantity after ':'
                    spillvolume = line.split(':')[-1].split('\n')[0]
                    # convert to float for "sort_values" to work properly
                    volume[oil]['volume'].append(float(spillvolume)) 
        big50_spills[oil] = pandas.DataFrame.from_dict(
            volume[oil]).sort_values('volume',ascending=False)[0:50]

In [ ]:
# %%time

# spills_dict = big50_spills.copy
# # if os.path.exists('/home/rmueller/projects/def-allen/rmueller/MIDOSS/analysis-rachael/notebooks/monte_carlo_dev/TopSpills50_df.csv'): 
# #     TopSpills = pandas.read_csv('/home/rmueller/projects/def-allen/rmueller/MIDOSS/analysis-rachael/notebooks/monte_carlo_dev/TopSpills50_df.csv')
# # else:
# ## Generate "TopSpills_df.csv"
# ## This section takes 10 minutes on 5210 Mb
# vars2plot= ['OilConcentration_3D','OilWaterColumnOilVol_3D']
# oil_type=[]
# spill_index=[]
# spill_volume=[]
# #variable_list = []
# mn={}
# mx={}
# mean={}
# for variable in vars2plot:
#     mn[variable] = []
#     mx[variable] = []
#     mean[variable] = []

# for oil in ['diesel','bunker']:
#     # loop through indices of the largest spills in the first 100 spill files
#     for idx in big50_spills[oil].index.values:
#         spill_xr=xarray.open_dataset(First100[oil][idx])
#         oil_type.append(oil)
#         spill_index.append(idx)
#         spill_volume.append(big50_spills[oil]['volume'][idx])
#         for variable in vars2plot:
#             print(f'[{oil}-{idx}-{variable}]')     
#             mn[variable].append(spill_xr[variable][:,39,:,:].min().values.item())
#             mx[variable].append(spill_xr[variable][:,39,:,:].max().values.item())
#             mean[variable].append(spill_xr[variable][:,39,:,:].mean().values.item())
#         spill_xr.close()
# TopSpills = {'oil_type':oil_type, 'spill_volume':spill_volume,
#     'MeanSfcConc':mean['OilConcentration_3D'], 'MinSfcConc':mn['OilConcentration_3D'], 
#     'MaxSfcConc':mx['OilConcentration_3D'], 'MeanSfcVol':mean['OilWaterColumnOilVol_3D'],
#     'MinSfcVol':mn['OilWaterColumnOilVol_3D'], 'MaxSfcVol':mx['OilWaterColumnOilVol_3D']}
# TopSpills_df = pandas.DataFrame(TopSpills)
# TopSpills_df.to_csv('TopSpills50_df.csv')

## plot up Concentration vs. Volume for 50 diesel and 50 bunker spills

In [ ]:
TopSpills = pandas.read_csv('/home/rmueller/projects/def-allen/rmueller/MIDOSS/analysis-rachael/notebooks/monte_carlo_dev/TopSpills50_df.csv')

In [ ]:
TopSpills[:5]

In [ ]:
TopSpills['OilFlag'] = TopSpills['oil_type']
TopSpills.loc[TopSpills['oil_type'] == 'bunker', 'OilFlag'] = 2
TopSpills.loc[TopSpills['oil_type'] == 'diesel', 'OilFlag'] = 1

In [ ]:
TopSpills[TopSpills['oil_type']=='diesel'].plot.scatter(x='MaxSfcVol',y='MaxSfcConc',
                                                        s=60, c='purple')
TopSpills[TopSpills['oil_type']=='bunker'].plot.scatter(x='MaxSfcVol',y='MaxSfcConc', 
                                                        s=60, c='green')
TopSpills[TopSpills['oil_type']=='diesel'].plot.scatter(x='MeanSfcVol',y='MeanSfcConc',
                                                        s=60, c='purple')
TopSpills[TopSpills['oil_type']=='bunker'].plot.scatter(x='MeanSfcVol',y='MeanSfcConc', 
                                                        s=60, c='green')
TopSpills['Vol2Conc'] = TopSpills['MaxSfcVol']/TopSpills['MaxSfcConc']   
ax_ratio = TopSpills.plot.scatter(x='OilFlag',y='Vol2Conc', s=60, c='blue')
ax_ratio.set_xticks((0,1,2,3))
ax_ratio.set_xticklabels(['','diesel','bunker',''])
ax_ratio.set_title('Ratio of Maximum Volume to Maximum Concentration')
TopSpills['Vol2Conc_mean'] = TopSpills['MeanSfcVol']/TopSpills['MeanSfcConc']
ax_ratio = TopSpills.plot.scatter(x='OilFlag',y='Vol2Conc_mean', 
                                                        s=60, c='blue')
ax_ratio.set_xticks((0,1,2,3))
ax_ratio.set_xticklabels(['','diesel','bunker',''])
ax_ratio.set_title('Ratio of Mean Volume to Mean Concentration')

